In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Kütüphaneleri ekliyoruz
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn import linear_model
import seaborn as sns
from scipy import stats
from scipy.stats import norm, skew 
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error

In [ ]:
#Datalarımızı tanımlıyoruz
sample_submission = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv")
test = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/test.csv")
train = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv")
items = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/items.csv")
item_categories = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv")
shops = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/shops.csv")

In [ ]:
train.item_cnt_day.plot()
plt.title("Günlük satılan ürün sayısı");

In [ ]:
target = train.item_cnt_day
train = train.drop(['item_price','item_cnt_day','date','date_block_num'],axis=1).select_dtypes(exclude=['object'])

In [ ]:
#Verilerimizi Train-Test olarak ayırıyoruz %80Traine-20Teste
train_X, test_X, train_y, test_y = train_test_split(train,target,test_size=0.20)

In [ ]:
#Modelimizi eğitiyoruz 
my_model = XGBRegressor(n_estimators=10, learning_rate=0.07)
my_model.fit(train_X, train_y, early_stopping_rounds=10, 
             eval_set=[(test_X, test_y)], verbose=False)

In [ ]:
#Boş olan hücreleri dolduruyoruz 
test = test.select_dtypes(exclude=['object'])
test.fillna( method ='ffill', inplace = True)
test_X = test
test_X = test_X.drop(['ID'],axis=1).select_dtypes(exclude=['object'])

In [ ]:
#Tahmin ediyoruz
predictions = my_model.predict(test_X)
predictions

In [ ]:
#Sonuçları csv dosyası haline getiriyoruz
my_submission = pd.DataFrame({'Id': test.ID, 'item_cnt_month': predictions})
my_submission.to_csv('submission.csv', index=False)